<a href="https://colab.research.google.com/github/HodaMemar/A3/blob/main/NER.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install -U spacy
!pip install -U scispacy

In [ ]:
!pip install spacy-transformers

In [ ]:
!pip install negspacy

In [ ]:
!pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz
 #https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.2.4/en_core_sci_sm-0.2.4.tar.gz
 #https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.0/en_ner_bc5cdr_md-0.5.0.tar.gz



#Reset Runtime

#load text

In [ ]:
cd /content/drive/MyDrive/PaperA3/Section


In [ ]:
import re

import scispacy
import spacy

from negspacy.negation import Negex
from negspacy.termsets import termset  
from scispacy.linking import EntityLinker
import pandas as pd

nlp = spacy.load("en_ner_bc5cdr_md")

#
ts = termset("en_clinical")
nlp.add_pipe("negex", config={"neg_termset": ts.get_patterns()})
nlp.add_pipe("scispacy_linker", config={"linker_name": "umls", "max_entities_per_mention": 1})

linker = nlp.get_pipe("scispacy_linker")
nlp.pipe_names

In [ ]:
def nlp_Entity(x):
      doc=nlp(x)
      entity = doc
      return entity

In [ ]:
def process_clinical_Notes (data,section):
#doc = nlp("Patient has  prostate cancer with metastatic disease to his bladder.")
#fmt_str = "{:<20}| {:<10}| {:<32}| {:<20}"
#print(fmt_str.format("Entity", "1st CUI", "Canonical Name", "Definition"))
    list_Cui=[]
    list_Exception=[]
    c=len(data)

    fmt_str = "{:<15}| {:<6}| {:<7}| {:<8}"
    print(fmt_str.format("token", "pos", "label", "parent"))

    for i in range(len(data)):
            text= str(data.iloc[i]['body_span_Wothout_line'])
            HADM_ID	=data.iloc[i]['HADM_ID']
            title_span=data.iloc[i]['title_span']
            category_Inner=	data.iloc[i]['category_Inner']
            CATEGORY=	data.iloc[i]['CATEGORY']
            DRG_CODE=	data.iloc[i]['DRG_CODE']
            GENDER=	data.iloc[i]['GENDER']
            AGE=	data.iloc[i]['AGE']
            CHARTDATE=	data.iloc[i]['CHARTDATE']
            TAA=	data.iloc[i]['TAA']

            doc = nlp(text)
            print(c-i)
            
            print("\n")
            for entity in doc.ents:
                if(len(entity._.kb_ents)>0):
                    first_cuid = entity._.kb_ents[0][0]
                    kb_entry = linker.kb.cui_to_entity[first_cuid]
                    list_Cui.append([HADM_ID,title_span,category_Inner,entity._.negex,entity.text, first_cuid, kb_entry.canonical_name,entity.label_,CATEGORY,DRG_CODE,
                                     GENDER,AGE,CHARTDATE,TAA])
                else:
                    list_Exception.append([HADM_ID,title_span,category_Inner,entity.text])
                    continue   



    df_cui=pd.DataFrame(list_Cui,columns=['HADM_ID','title_span','category_Inner','negex','entity_text', 'first_cuid', 'canonical_name','label',
                                          'CATEGORY','DRG_CODE',
                                     'GENDER','AGE','CHARTDATE','TAA']) 
    #df_Exception=pd.DataFrame(list_Exception,columns=['HADM_ID','title_span','category_Inner','entity_text']) 
    
    filename="/content/drive/MyDrive/PaperA3/CUI_"+"_"+str(section)+".csv"
    #filename_exeption="/content/drive/MyDrive/PaperA3/CUI_Exeption_"+str(chunk_index)+".csv" 

    df_cui.to_csv(filename)
    #df_Exception.to_csv(filename_exeption)

In [ ]:
      %%time
import pandas as pd

for section in (13,14) :

    filename="/content/drive/MyDrive/PaperA3/Section/section_"+str(section)+".csv"
    print(filename)
    df=pd.read_csv(filename)
    process_clinical_Notes(df,section)

#data = pd.read_csv('/content/drive/MyDrive/PaperA3/Notes_DRG_Valid.csv', error_bad_lines=False)
